In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.feature_selection import RFE, RFECV
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, f1_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
cols1=['impression_time','user_id','app_code','os_version','is_4G','is_click']
cols2=['device_type','user_id','item_id']
cols3=['impression_time','user_id','app_code','os_version','is_4G']

In [5]:
train=pd.read_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Train\train.csv' )

In [6]:
View_log=pd.read_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Train\view_log.csv')

In [7]:
item_data=pd.read_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Train\item_data.csv')

In [8]:
test=pd.read_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Test\test.csv')

In [9]:
train.sample(n=10)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
6890,ec5c931cc3eb9fcc8a46b364c0f69e51,2018-11-16 22:54:00,13946,207,old,1,0
43690,9d63220a4075c63116682c1c51048a64,2018-11-23 15:37:00,80619,386,latest,1,0
72838,2a59997ba5ba15483d2ac0f8e5009d2a,2018-11-21 22:27:00,58506,409,latest,1,0
76719,54d35266ad22814ef605d764c305cd46,2018-11-27 16:31:00,53431,386,old,0,0
46774,13d29e6e8cf1bcf427da3e7bd696a73f,2018-11-19 16:19:00,36267,244,old,0,0
105688,17df7bcb4dc3cd1953798284a544e0e4,2018-11-30 09:50:00,42722,465,old,0,0
93805,5615e21075a0d5ba1af059faf59e4c88,2018-11-29 02:15:00,77617,385,intermediate,0,0
98144,60cffc4164b276ee0ebfcf7a98bcd7e2,2018-11-29 16:26:00,63904,242,intermediate,0,0
194749,fb9a5783a6d3142d64dff352ca5b2e21,2018-12-05 03:07:00,65496,386,latest,0,0
160741,6d0ed68d68188d8275fc2f5108299d54,2018-12-01 20:31:00,61898,190,old,1,0


In [10]:
train['impression_time']=pd.to_datetime(train['impression_time'])
test['impression_time']=pd.to_datetime(test['impression_time'])

In [11]:
train['impression_time_hour']=train.impression_time.dt.hour
test['impression_time_hour']=test.impression_time.dt.hour

In [13]:
train['impression_time_day']=train.impression_time.dt.dayofweek
test['impression_time_day']=test.impression_time.dt.dayofweek

In [14]:
print(train.columns)
print(test.columns)

Index(['impression_id', 'impression_time', 'user_id', 'app_code', 'os_version',
       'is_4G', 'is_click', 'impression_time_hour', 'impression_time_day'],
      dtype='object')
Index(['impression_id', 'impression_time', 'user_id', 'app_code', 'os_version',
       'is_4G', 'impression_time_hour', 'impression_time_day'],
      dtype='object')


In [15]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,impression_time_hour,impression_time_day
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,0,3
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,0,3
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,0,3
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,0,3
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,0,3


In [16]:
def convert_os(ver):
    if ver=='latest':
        return 1
    elif ver=='intermediate':
        return 2
    else:
        return 3

In [17]:
train['os_version']=train['os_version'].apply(convert_os)

In [18]:
test['os_version']=test['os_version'].apply(convert_os)

In [19]:
test.sample(n=10)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,impression_time_hour,impression_time_day
24562,77e3bc45edb244685fb1ad1e19e50ce6,2018-12-15 16:02:00,7651,386,1,1,16,5
84744,631d4c1a11469f7c2850d3996301b2f1,2018-12-18 11:53:00,42270,386,2,1,11,1
76930,808d2f7f8a16b5600b2060c1820e81a4,2018-12-17 20:16:00,17782,202,1,0,20,0
57034,e7a2df72656b9b925762b75fec328361,2018-12-16 00:06:00,31297,381,3,0,0,6
90226,bb4fc4b52d4e6822dc84f9e07123b9ee,2018-12-18 18:16:00,7370,514,1,0,18,1
32863,2ef9f51d8e7a2e639ad361c8071407ea,2018-12-14 11:00:00,52082,385,3,0,11,4
64038,6debc93e59bbb200ab487aa2953edebe,2018-12-17 00:45:00,47408,32,2,0,0,0
77805,e9f25479c9c234e8703bcc95cd1abf49,2018-12-18 12:49:00,53389,463,2,0,12,1
4804,21cd27c4765b3c36c29df276791fb729,2018-12-13 09:21:00,46357,207,2,0,9,3
60207,bab5cd71381b17cee89bdfa1e8b0a69f,2018-12-16 13:40:00,25677,44,3,0,13,6


In [20]:
View_log.dtypes

server_time    object
device_type    object
session_id      int64
user_id         int64
item_id         int64
dtype: object

In [21]:
View_log['server_time']=pd.to_datetime(View_log['server_time'])

In [22]:
View_log['server_time_day']=View_log.server_time.dt.day

In [23]:
View_log['server_time_dayofweek']=View_log.server_time.dt.dayofweek

In [24]:
visit_on_day=View_log.groupby(['user_id','server_time_dayofweek']).item_id.count().reset_index()

In [25]:
visit_in_month=View_log.groupby('user_id').item_id.count().reset_index()

In [34]:
new_df=View_log[['user_id','device_type']]

In [35]:
new_df.shape

(3118622, 2)

In [36]:
new_df.drop_duplicates(inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
new_df['device_type'].value_counts()

android    89154
iphone        61
web           24
Name: device_type, dtype: int64

In [39]:
def convert_device(ver):
    if ver=='android':
        return 1
    elif ver=='iphone':
        return 2
    else:
        return 3

In [40]:
new_df['device_type']=new_df.device_type.apply(convert_device)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
new_df['device_type'].value_counts()

1    89154
2       61
3       24
Name: device_type, dtype: int64

In [42]:
print(visit_in_month.dtypes)
print(visit_on_day.dtypes)
print(new_df.dtypes)

user_id    int64
item_id    int64
dtype: object
user_id                  int64
server_time_dayofweek    int64
item_id                  int64
dtype: object
user_id        int64
device_type    int64
dtype: object


In [43]:
df1=pd.merge(new_df, visit_in_month, how='inner', on=['user_id'])

In [44]:
df1.rename(columns={"item_id":'item_count'} ,inplace=True)

In [45]:
visit_on_day.rename(columns={"server_time_dayofweek":'impression_time_day',"item_id":'visit_count'}, inplace=True)

In [46]:
train1=pd.merge(train, df1, how='left', on='user_id')
test1=pd.merge(test, df1, how='left', on='user_id')

In [47]:
train2=pd.merge(train1, visit_on_day, how='left', on=['user_id','impression_time_day'])
test2=pd.merge(test1, visit_on_day, how='left', on=['user_id','impression_time_day'])

In [48]:
train2.fillna(value=0, inplace=True)
test2.fillna(value=0, inplace=True)

In [49]:
item_data.columns

Index(['item_id', 'item_price', 'category_1', 'category_2', 'category_3',
       'product_type'],
      dtype='object')

In [50]:
view1=pd.merge(View_log, item_data, how='left', on='item_id')

In [51]:
category1=view1.groupby(['user_id','category_1']).category_2.count().reset_index()
category2=view1.groupby(['user_id','category_2']).category_3.count().reset_index()
category3=view1.groupby(['user_id','category_3']).product_type.count().reset_index()

In [52]:
category4=category1.groupby('user_id').max().reset_index()
category5=category2.groupby('user_id').max().reset_index()
category6=category3.groupby('user_id').max().reset_index()

In [53]:
print(category4.head())
print(category5.head())
print(category6.head())

   user_id  category_1  category_2
0        0        17.0           9
1        1        15.0           2
2        2        17.0          31
3        3        14.0           3
4        4        14.0           1
   user_id  category_2  category_3
0        0        79.0           6
1        1        78.0           2
2        2        78.0          11
3        3        62.0           3
4        4        61.0           1
   user_id  category_3  product_type
0        0       308.0             6
1        1       277.0             2
2        2       336.0             7
3        3       157.0             3
4        4       159.0             1


In [54]:
category4.rename(columns={"category_2":'count1'}, inplace=True)
category5.rename(columns={"category_3":'count2'}, inplace=True)
category6.rename(columns={"product_type":'count3'}, inplace=True)

In [55]:
category=category4.merge(category5, on='user_id').merge(category6, on='user_id')

In [56]:
train3=pd.merge(train2, category, how='left', on='user_id')
test3=pd.merge(test2, category, how='left', on='user_id')

In [57]:
train3.fillna(value=0, inplace=True)
test3.fillna(value=0, inplace=True)

In [58]:
train3=train3.astype({"visit_count":np.int64, "category_1":np.int64, "count1":np.int64, "category_2":np.int64, "count2":np.int64,
              "category_3":np.int64, "count3":np.int64}, inplace=True)

In [59]:
train3.columns

Index(['impression_id', 'impression_time', 'user_id', 'app_code', 'os_version',
       'is_4G', 'is_click', 'impression_time_hour', 'impression_time_day',
       'device_type', 'item_count', 'visit_count', 'category_1', 'count1',
       'category_2', 'count2', 'category_3', 'count3'],
      dtype='object')

In [60]:
col1=['app_code','user_id','impression_time_hour','impression_time_day','os_version','device_type','item_count','visit_count',
     'category_1','count1','category_2','count2','category_3','count3','is_4G','is_click']
col2=['app_code','user_id','impression_time_hour','impression_time_day','os_version','device_type','item_count','visit_count',
     'category_1','count1','category_2','count2','category_3','count3','is_4G']

In [61]:
train_final=train3[col1]
test_final=test3[col2]

In [62]:
vif=pd.DataFrame()
vif['vif_factor']=[variance_inflation_factor(train_final.values, i) for i in range(train_final.shape[1])]
vif['feature']=train_final.columns
vif

,vif_factor,feature
0,4.379664,app_code
1,3.983091,user_id
2,4.499692,impression_time_hour
3,3.007309,impression_time_day
4,5.282415,os_version
5,51.340109,device_type
6,15.648422,item_count
7,3.610307,visit_count
8,43.271036,category_1
9,32.740039,count1


In [87]:
train_final.shape

(237892, 16)

In [76]:
clicked_on_app=train_final.groupby('app_code').is_click.sum().reset_index()

In [83]:
ind=clicked_on_app.query('is_click==0').index
clicked_on_app.drop(ind, axis=0,inplace=True)

In [85]:
train4=pd.merge(train_final, clicked_on_app, how='right', on='app_code')

In [89]:
train4.columns

Index(['app_code', 'user_id', 'impression_time_hour', 'impression_time_day',
       'os_version', 'device_type', 'item_count', 'visit_count', 'category_1',
       'count1', 'category_2', 'count2', 'category_3', 'count3', 'is_4G',
       'is_click_x', 'is_click_y'],
      dtype='object')

In [90]:
X=train4[col2]
y=train4['is_click_x']
xTest=test_final[col2]

In [109]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.25)

In [393]:
features = [11,12,13,14,15]
for n in features:
    model = DecisionTreeClassifier()
    rfe = RFE(model, n_features_to_select = n)
    rfe = rfe.fit(X_train, y_train)
    cols = X_train.columns.tolist()
    sel_feat = pd.DataFrame({"cols": cols, "support": rfe.support_, "rank": rfe.ranking_})
    print("Top features: ", n)
    print(rfe.score(X_train, y_train))
    print(f1_score(y_test, rfe.predict(X_test)))
    print(sel_feat[sel_feat['rank'] == 1]['cols'].unique(), '\n')

Top features:  11
0.9911836743844545
0.12155278412085016
['app_code' 'impression_time_hour' 'impression_time_day' 'os_version'
 'item_count' 'count1' 'category_2' 'count2' 'category_3' 'count3' 'is_4G'] 

Top features:  12
0.9911892791686984
0.11423487544483987
['app_code' 'impression_time_hour' 'impression_time_day' 'os_version'
 'item_count' 'category_1' 'count1' 'category_2' 'count2' 'category_3'
 'count3' 'is_4G'] 

Top features:  13
0.9911892791686984
0.11687621746059854
['app_code' 'impression_time_hour' 'impression_time_day' 'os_version'
 'device_type' 'item_count' 'category_1' 'count1' 'category_2' 'count2'
 'category_3' 'count3' 'is_4G'] 

Top features:  14
0.9911892791686984
0.11547998583067658
['app_code' 'impression_time_hour' 'impression_time_day' 'os_version'
 'device_type' 'item_count' 'category_1' 'count1' 'category_2' 'count2'
 'category_3' 'count3' 'is_4G'] 

Top features:  15
0.9911892791686984
0.11663991439272338
['app_code' 'impression_time_hour' 'impression_time_d

In [110]:
y_test.sum()

2688

In [68]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
res=knn.predict(X_test)

In [99]:
naiv=DecisionTreeClassifier()
naiv.fit(X_train, y_train)
nvs=naiv.predict(X_test)

In [111]:
nv=GaussianNB()
nv.fit(X_train, y_train)
res=nv.predict(X_test)

In [112]:
res.sum()

3652

In [113]:
confusion_matrix(y_true=y_test, y_pred=res)

array([[53040,  3383],
       [ 2419,   269]], dtype=int64)

In [114]:
roc_auc_score(y_test, res)

0.5200582930709192

In [328]:
sample=pd.read_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Results\sample_submission.csv')

In [429]:
sample['is_click']=pd.DataFrame(res, columns=['is_click'])

In [430]:
sample.to_csv(r'C:\Users\Niraj\Desktop\Ana Hackathon\WNS\Results\result7.csv', index=False, header=True)